# Hello world

In [3]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="text-davinci-002",
                                        prompt =  texts,
                                        temperature = 0.6,
                                        top_p = 1,
                                        max_tokens = 64,
                                        frequency_penalty = 0,
                                        presence_penalty = 0)
    return print(response.choices[0].text)

recipe = 'Provide a cooking recipe based on the following ingredients: \\n \nApple \
          \n \nFlour \
          \n \nChicken \
          \n \nSalt'

GPT_Completion(recipe)



Apple Chicken

Ingredients:

1 apple, cored and sliced
1 chicken breast, cut into small pieces
1/4 cup flour
1/4 teaspoon salt

Instructions:

1. Preheat oven to 375 degrees F.
2. In a medium bowl, combine


# Generate learning dataset

## Generate problems

In [3]:
import os
import filecmp

## 8000 data points should improve performance between 2x and 4x

%run generator/pddl_confgen.py 4 8000 -r 24 -co
!mv -v generator/problems/conditional/* dataset/problems > /dev/null

## clean duplicates
print("Cleaning duplicates...")

problems = sorted(os.listdir('./dataset/problems/'))

duplicates = 0

for i, problem_i in enumerate(problems):
    for j, problem_j in enumerate(problems):
        try:
            if (problem_i != problem_j) and filecmp.cmp(problem_i, problem_j):
                duplicates += 1
                os.remove(problem_j)
        except FileNotFoundError:
            continue
            
print(str(duplicates) + " duplicates deleted")

Writing conditional problem files to /home/alessio/planning/teriyaki/generator/problems/conditional...
Done.
All operations completed. Happy planning!
Cleaning duplicates...
0 duplicates deleted


## Generate plans with Probe

In [ ]:
import os
import subprocess

problems = sorted(os.listdir('./dataset/problems/'))

for i, problem in enumerate(problems):
    result = subprocess.run(['./generator/probe', 
                             '-d', './dataset/domains/domain_m_25_B.pddl', 
                             '-i', './dataset/problems/' + problem,
                             '-o', './dataset/plans/m_25_B_plan-' + str(problem.split('_',2)[2:][0])], 
                             stdout=subprocess.PIPE)
    print("Planning --- " + str(round((i/8000), 1)) +"%", end = '\r')


## Convert Probe plans to IPC format

In [38]:
import os

plans = sorted(os.listdir('./dataset/plans/'))

for plan in plans:
    probe = open('./dataset/plans/' + plan, "r")
    val = open('./dataset/plans/' + plan.split(".")[0] + ".plan", 'w+')
    replacement = ""
    # using the for loop
    for i, line in enumerate(probe):
        line = line.lower()
        line = str((i + 1)/1000) + "00: " + line
        val.write(line)

    probe.close()
    os.remove('./dataset/plans/' + plan)
    val.close()
    
print("Plans converted to IPC format")

Plans converted to IPC format
